# Generating Reports using SoS

There are multiple ways to generate reports from the execution of SoS workflows. This tutorial introduces some basic methods but you can certainly be more creative.

## A summarization step

You can write a report at the end of the workflow that summarizes the results of previous steps. For example, in the following example, the `report` action summarizes previous steps and writes a report to the standard output.

In [1]:
%sandbox
%run

[10]
output: 'a.jpg'
R: expand=True
    jpeg({output!r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[100]
input: 'a.jpg', 'a.txt'
with open('a.txt') as a:
    res = a.read()

report: expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {res}
   

echo "100" > a.txt

* Figure
![figure](a.jpg)
* result
100



It is a pretty bad idea to write report to standard output because other actions can also write to it. You should therefore use the `output` option of the `report` action to specify an output file of the `report` action.

In [2]:
%sandbox
%preview summary.md
%run

[10]
output: 'a.jpg'
R: expand=True
    jpeg({output!r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[counts: shared='counts']
input: 'a.txt'
with open('a.txt') as a:
    counts = a.read()

[100]
input: 'a.jpg'
depends: sos_variable('counts')

report: output='summary.md', expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {counts}
   

INFO: Resolving [sos_variable("counts")] objects from 3 nodes


echo "100" > a.txt



Instead of outputting reports in `.md` format and rendering them outside of SoS, you can also render them inside SoS using action `pandoc` or `Rmarkdown`. For example,

In [3]:
%sandbox
%preview summary.html
[10]
output: 'a.jpg'
R: expand=True
    jpeg({output!r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

[20]
output: 'a.txt'
run:
    echo "100" > a.txt

[counts: shared='counts']
input: 'a.txt'
with open('a.txt') as a:
    counts = a.read()

[100]
input: 'a.jpg'
depends: sos_variable('counts')

pandoc: output='summary.html', expand=True
    * Figure
    ![figure](a.jpg)
    * result
    {counts}

## Reporting to multiple output files

If you have a large workflow with several steps or modules, it makes sense to write several reports and join them at last. For example, in the following example, an output file is specified for each `report` and a summary step is used at the end to collect and process them.

In [4]:
%sandbox --dir ~/tmp
%preview summary.html
[10]
output: 'a.jpg'
R: expand=True
    jpeg({output!r})
    cars <- c(1, 3, 6, 4, 9)
    barplot(cars)

report: output='figure.md', expand=True
    * Figure
    ![figure]({output})    

[20]
output: 'a.txt'
counts = 100
run: expand=True
    echo "{counts}" > a.txt

report: output='result.md', expand=True
    * result
    {counts}

[100]
pandoc: input=['figure.md', 'result.md'], output='summary.html'
    Final report

You will notice that both `script` and `input` are specified to action `pandoc`. In this case the `script` is put before the content of each `input` files, making it a perfect place to write headers and summaries.